# Iterative Dichotomiser 3 (ID3)

<br>

## Introduction

<br>
One approach to the induction task above would be to generate all possible decision trees that correctly classify the training set and to select the simplest of them. The number of such trees is finite but very large, so this approach would only be feasible for small induction tasks. 

<br>
Most algorithms that have been developed for learning decision trees are variations on a core algorithm that employs a top-down, greedy search through the space of possible decision trees. This algorithm was designed to operate in the opposite end of the spectrum, where there are many attributes and the training set contains many objects, but where a reasonably good decision tree is required without much computation. 

<br>
This approach is exemplified by the ID3 algorithm (<a href="https://bit.ly/2JCjrKM">Quinlan, 1986</a>) and its successor C4.5 (<a href="https://bit.ly/2HIsjwG">Quinlan, 1993</a>), which form the primary focus of the current and following notebook. ID3 has been found to construct simple decision trees, but it cannot guarantee that better trees have not been overlooked. 

## The Algorithm

<br>
The ID3 algorithm learns decision trees by constructing them top-down, beginning with the original set $ \ S \ $ as the root node. On each iteration of the algorithm : 

<br>
<ol type="i">
    <li>
        every unused attribute is evaluated using a statistical test to determine how well it alone classifies the training
        examples
    </li>
    <br>
    <li>
        the best attribute is selected and used as the test at the root node of the tree
    </li>
    <br>
    <li>
        the set $ \ S \ $ is then split by the selected attribute to produce subsets of the data, each of which will
        constitute a descendant of the parent node
    </li>
    <br>
    <li>
        <b>the entire process continues recursively on each of the descendant nodes (or subsets)</b>, determining the best
        attribute to use at that point in the tree <b>considering only attributes never selected before</b>
    </li>
</ol>

<br>
<b>Note</b> : the technique we just described performs a greedy search for an acceptable decision tree, in which the algorithm never backtracks to reconsider earlier choices.


<table>
    <tr>
        <td width="33.3%"><img src="images/greedy_search_1.png" alt=""></td>
        <td width="33.3%"><img src="images/greedy_search_2.png" alt=""></td>
        <td width="33.3%"><img src="images/greedy_search_3.png" alt=""></td>
    </tr>   
</table>

<br>
<center><b>Figure 1</b> : top-down, greedy search through the space of possible decision trees</center>

<br>
Recursion on a subset (or internal node) may stop in one of these cases :

<br>
<ul style="list-style-type:square">
    <li>
        every element in the subset belongs to the same class; in this case the node is turned into a leaf and labelled with the
        class of the examples
    </li>
    <br>
    <li>
        the examples still do not belong to the same class but there are no more attributes to be selected, in this case the
        node is turned into a leaf and labelled with the most common class of the examples in the subset
    </li>
    <br>
    <li>
        there are no examples in the subset, this happens when no example in the parent set was found to be matching a specific
        value of the selected attribute; in this case a leaf is created, and labelled with the most common class of the examples
        in the parent set
    </li>
</ul>

### Iterative Variation

<br>
Empirical evidence suggests that, rather than forming a tree directly from the entire training set, a correct decision tree is usually found more quickly using an iterative version of the core algorithm :  

<br>
<ol type="i">
    <li>
        a subset of the training set called the window is chosen at random, 
        and a decision tree is formed from it in such a way that this tree correctly classifies all objects in the window
    </li>
    <br>
    <li>
        all other objects in the training set are then classified using the tree. If the tree gives the correct answer for all
        these objects then it is correct for the entire training set and the process terminates; if not, a selection
        of the incorrectly classified objects is added to the window and the process continues
    </li>
</ol>

<br>
Correct decision trees have been found after only a few iterations for training sets of up to thirty thousand objects described in terms of up to 50 attributes. 

## Attribute Selection Metrics

<br>
The central choice in the ID3 algorithm is selecting which attribute to test at each node in the tree; we would like to select the attribute that is most useful for classifying examples, but what is a good quantitative measure of the worth of an attribute?


### Entropy

<br>
In Information Theory, (information) <b>entropy is defined as the average amount of information produced by a stochastic source of data</b> or, equivalently, as a measure of unpredictability of the state of a system.

<br>
<b>Example</b> : to get an intuitive understanding of these terms, we consider the example of a <b>coin toss</b>; assuming the probability of heads is the same as the probability of tails, then the entropy of the coin toss is as high as it could be. This is because there is no way to predict the outcome of the coin toss ahead of time: if we have to choose, the best we can do is predict that the coin will come up heads, and this prediction will be correct with probability 1/2. Such a coin toss has one bit of entropy since there are two possible outcomes that occur with equal probability, and learning the actual outcome contains one bit of information. In contrast, a coin toss using a coin that has two heads and no tails has zero entropy since the coin will always come up heads, and the outcome can be predicted perfectly.

Claude Shannon defined the entropy $ \ \mathrm{H} \ $ of a discrete random variable $ \ X \ $ with probability mass function $ \ P(X) \ $ and information content $ \ \mathrm{I}(X) \ $ as below, where the measure of information entropy associated with each possible data value is the negative logarithm of the probability mass function for the value :

<br>
$
    \quad
    \begin{align}
        \mathrm{H}(X) 
        &= 
        \newline
        &= \mathbf{E} \left[ \ \mathrm{I}(X) \ \right] = \mathbf{E} \left[ \ - \ln \mathrm{P}(X) \ \right]
        \newline
        &= 
                \sum _{i=1}^{n} {\mathrm{P}(x_{i}) \ \mathrm{I}(x_{i})} 
            = - \sum _{i=1}^{n} {\mathrm{P}(x_{i}) \ \log _{b} \ \mathrm {P}(x_{i})}
            \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad 
            & [\textbf{E1}] 
    \end{align}
$

<br>
In the case of $ \ \mathrm{P}(x_{i}) = 0 \ $ for some $i$, the value of the corresponding summand 0 logb(0) is taken to be 0, which is consistent with the limit:

<br>
$ \quad \lim _{p\to 0+} p\log(p) = 0 $

<br>
Common values for base of the logarithm are 2, Euler's number $ \ e \ $, and 10; the corresponding units of entropy are the binary digits (bits), natural digits (nats), and decimal digits (decits).

<b>Diversity Index</b>

<br>
It has been shown that Shannon Entropy can be used as a diversity index : <b>a quantitative measure that reflects how many different types there are in a dataset and simultaneously takes into account how evenly the basic entities are distributed among those types</b>. 

<b>Example</b> : to illustrate, suppose $ \ S \ $ is a collection of 14 examples of some boolean
concept (positive/negative, red/blue), including 9 red and 5 blue examples; then the entropy of the collection $ \ S \ $ relative to this boolean classification is :

<br>
$
    \quad
    \mathrm{H}(S) \quad = \quad - \ \tfrac{9}{14} \log _{2}\tfrac{9}{14} \quad - \tfrac{5}{14} \log _{2}\tfrac{5}{14} 
    \quad = \quad 0.94
$

<br>
Notice that the entropy is 0 if all members of $ \ S \ $ belong to the same class, and the entropy is 1 when the collection contains an equal number of positive and negative examples.

### Information Gain

<br>
Given entropy as a measure of the impurity in a collection of training examples, we can now define a measure of the effectiveness of an attribute in classifying the training data. The measure we will use, called <b>information gain, is simply the expected reduction in entropy caused by partitioning the examples according to some attribute $ A $</b>.

<br>
More precisely, the information gain $ \ Gain(S, A) \ $ of an attribute $ \ A \ $ relative to a collection of examples $ \ S \ $ is defined as :

<br>
$
    \quad
    Gain(S, A) \quad = \quad
        \mathrm{H}(S) 
        \ - \ \sum _{\ v \ \in \ values(A)} \dfrac{\mid \ S_v \ \mid}{\mid \ S \ \mid} \ \mathrm{H}(S_v)
        \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad
        [\textbf{E2}] 
$

where $ values(A) $ is the set of all possible values for attribute $ \ A \ $, and $ \ S_v \ $ is the subset of $ \ S \ $ for which attribute $ \ A \ $ has value $ \ v \ $ ($ \ S_v = \{s \in S \mid A(s) = v\} \ $).

<br> <br>
The first term in <b>E2</b> is just the entropy of the original collection, and the second term is the sum of the entropies of each subset weighted by the fraction of examples that belong to the subset. Equivalently, <b>the second term represents the expected value of the entropy after the original collection $ \ S \ $ is partitioned using attribute $ \ A \ $ </b>.

<br>
In other words, the information gain is the information provided about the target function value, given the value of some
other attribute $ \ A \ $. The value of $ \ Gain(S, A) \ $ is the number of bits saved when encoding the target value of an arbitrary member of the collection $ \ S \ $, by knowing the value of attribute $ \ A \ $.

<br>
<a id='information_gain'>
    <img src="images/information_gain.jpg" alt="outlook" width="50%" height="50%">
</a>

<br>
<center><b>Figure 2</b> : use of information gain to evaluate the relevance of attributes in ID3</center>

<b>Example</b> : suppose $ \ S \ $ is a collection of days described by attributes including Wind, which can have the values Weak or Strong. As before, assume the collection is composed of 14 examples, 9 labelled as positive and 5 as negative. Now suppose that, of this training set, 6 of the positive and 2 of the negative examples have Wind = Weak, and the remainder have Wind = Strong. The information gain due to sorting the original collection by the attribute Wind may then be calculated as :

<br>
$
    \quad
    \begin{align}
        Gain(S, Wind) 
        &=
            \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad 
            \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad 
            & \text{by } \textbf{E2} 
        \newline
        &=
            \mathrm{H}(S) 
            \ - \ \sum _{\ v \ \in \ Wind} \dfrac{\mid \ S_v \ \mid}{\mid \ S \ \mid} \ \mathrm{H}(S_v)
        \newline
        &= 
            0.94 \ - \ \left[ \ \tfrac{8}{14} \mathrm{H}(S_{weak})  - \tfrac{6}{14} \mathrm{H}(S_{strong})  \ \right]
            & \text{by } \textbf{E1} 
        \newline
        &= 
            0.94 
            \ - \ \tfrac{8}{14} \left[ - \ \tfrac{6}{8} \log _{2}\tfrac{6}{8} - \tfrac{2}{8} \log _{2}\tfrac{2}{8} \ \right]
            \ - \ \tfrac{6}{14} \left[ - \ \tfrac{3}{6} \log _{2}\tfrac{3}{6} - \tfrac{3}{6} \log _{2}\tfrac{3}{6} \ \right]
        \newline
        &= 0.048
    \end{align}
$

<b>Drawbacks</b>

Although information gain is usually a good measure for deciding the relevance of an attribute, it is not perfect. <b>A notable problem occurs when information gain is applied to attributes that can take on a large number of distinct values</b>. For example, suppose that we are building a decision tree for some data describing the customers of a business. Information gain is often used to decide which of the attributes are the most relevant, so they can be tested near the root of the tree. One of the input attributes might be the customer's credit card number. This attribute has a high mutual information, because it uniquely identifies each customer, but we do not want to include it in the decision tree: deciding how to treat a customer based on their credit card number is unlikely to generalize to customers we haven't seen before.

<br>
<b>Information gain ratio is sometimes used instead; this biases the decision tree against considering attributes with a large number of distinct values</b>. However, <b>attributes with very low information values then appeared to receive an unfair advantage</b>.

## ID3 Properties and Limitations 

<br>
As with other inductive learning methods, ID3 can be characterized as searching a space of hypotheses for one that fits the training examples; the hypothesis space searched by ID3 is the set of possible decision trees. ID3 performs a top-down, simple-to-complex search through this hypothesis space, beginning with the empty tree, then considering progressively more elaborate hypotheses in search of a decision tree that correctly classifies the training data.

<br>
By viewing ID3 in terms of its search space and search strategy, we can get some insight into its capabilities and limitations.


<br>
<b>Completeness</b>

ID3 hypothesis space of all decision trees is a complete space of finite discrete-valued functions, relative to the available attributes. Because every finite discrete-valued function can be represented by some decision tree, ID3 avoids one of the major risks of methods that search incomplete hypothesis spaces (such as methods that consider only conjunctive hypotheses): that the
hypothesis space might not contain the target function.

<br>
<b>Singularity</b>

ID3 maintains only a single current hypothesis as it searches through the space of decision trees. By determining only a single hypothesis, ID3 loses the capabilities that follow from explicitly representing all consistent hypotheses. For example, it does not have the ability to determine how many alternative decision trees are consistent with the available training data.

<br>
<b>BackTracking</b>

ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it never backtracks to reconsider this choice. Therefore, it is susceptible to the usual risks of hill-climbing search without backtracking : converging to locally optimal solutions that are not globally optimal. In the case of ID3, a locally optimal solution corresponds to the decision tree it selects along the single search path it explores. However, this locally optimal solution may be less desirable than trees that would have been encountered along a different branch of the search.

<br>
<b>Robustness to noise</b>

ID3 uses all training examples at each step in the search to make statistically based decisions regarding how to refine its current hypothesis. One advantage of using statistical properties of all the examples (e.g. information gain) is that the resulting search is much less sensitive to errors in individual training examples. ID3 can be easily extended to handle noisy training data by modifying its termination criterion to accept hypotheses that imperfectly fit the training data.

<br>
<b>Discretization</b>

ID3 is harder to use on continuous data : if the values of any given attribute is continuous, then there are many more places to split the data on this attribute, and searching for the best value to split by can be time consuming.

<br>
<b>Overfitting</b>

ID3 can overfit to the training data; to avoid overfitting, smaller decision trees should be preferred over larger ones. This algorithm usually produces small trees, but it does not always produce the smallest possible tree.


## Inductive Bias in Decision Tree Learning

<br>
In the first notebook of this series regarding decision trees we referred to the idea of induction as the ability to generalize beyond the training set, i.e. to construct a decision tree that correctly classifies not only objects from the training set but other (unseen) objects as well. 

<br>
We now define the <b>inductive bias</b> as the policy by which an algorithm generalizes from observed training examples to unseen instance or, more formally, as <b>the set of assumptions that, together with the training data, deductively justify the classifications assigned by the learner to future instances</b>.

<br>
Given a collection of training examples, there are typically many decision trees consistent with these examples. Describing the inductive bias of ID3 therefore consists of describing the basis by which it chooses one of these consistent hypotheses over the others.

<br>
ID3 chooses the first acceptable tree it encounters in its simple-to-complex, hill-climbing search through the space of possible trees. Because of the subtle interaction between the attribute selection heuristic used by ID3 (information gain) and the particular training examples it encounters, it is difficult to characterize precisely the inductive bias exhibited by ID3. However, we can approximately characterize its bias by saying that <b>the ID3 search strategy</b>:

<br>
<ul style="list-style-type:square">
    <li>
         <b>selects in favor of shorter trees over longer ones</b>
    </li>
    <br>
    <li>
        <b>selects in favor of trees that place high information gain attributes close to the root over those that do not</b>
    </li>
</ul>

<br>
<b>Note</b> : ID3 searches <i>incompletely</i> through a <i>complete hypothesis space</i> (i.e., one capable of expressing
any finite discrete-valued function), from simple to complex hypotheses, until it finds a decision tree consistent with the data. Its inductive bias is solely a consequence of the ordering of hypotheses by its search strategy, the hypothesis space introduces no additional bias.

### Preference Bias vs Restriction Bias

<br>
The inductive bias of ID3 is thus a <i>preference</i> for certain hypotheses over others, with no hard restriction on the hypotheses that can be eventually enumerated. This form of bias is typically called a <i>preference bias</i> (or search bias). In contrast, a bias in the form of categorical restrictions on the set of hypotheses considered is typically called a <i>restriction bias</i> (or language bias).

<br>
Given that some form of inductive bias is required in order to generalize beyond the training data, which type of inductive bias shall we prefer, a preference bias or a restriction bias? Typically, a preference bias is more desirable than a restriction bias, because it allows the learner to work within a complete hypothesis space that is assured to contain the unknown target function. In contrast, a restriction bias that strictly limits the set of potential hypotheses is generally less desirable, because it introduces the possibility of excluding the unknown target function altogether. 

## Considerations

<br>
Decision trees provide a practical method for learning <b>discrete-valued functions</b>. The ID3 family of algorithms infers decision trees by growing them from the root downward, greedily selecting the next best attribute for each new decision branch added to the tree.

<br>
ID3 searches a <b>complete hypothesis space (i.e. the space of decision trees can represent any discrete-valued function defined over discrete-valued instances)</b>; it thereby avoids the major difficulty associated with approaches that consider only restricted sets of hypotheses : that the target function might not be present in the hypothesis space.

<br>
The <b>inductive bias implicit in ID3 includes a preference for smaller trees</b>; its search through the hypothesis space grows the tree only as large as needed in order to classify the available training examples.

<br>
Overfitting the training data is an important issue in decision tree learning; <b>post-pruning methods are therefore important to avoid overfitting in decision tree learning</b> (and other inductive inference methods that employ a preference bias).

<br>
A large variety of <b>extensions to the basic ID3 algorithm has been developed</b> by different researchers. These include methods for post-pruning trees, handling real-valued attributes, accommodating training examples with missing attribute values, incrementally refining decision trees as new training examples become available, using attribute selection measures other than
information gain, and considering costs associated with instance attributes.


## References

<br>
<ul style="list-style-type:square">
    <li>
         Tom Mitchell - 
         <a href="https://bit.ly/2IYw6Yc">
         Machine Learning</a>        
    </li>
    <br>
    <li>
        J. Ross Quinlan - 
        <a href="https://bit.ly/2JCjrKM">
        Induction of Decision Trees</a>  
    </li>
    <br>
    <li>
         Wikipedia - 
         <a href="https://bit.ly/2g0Mz24">
         Decision Tree Learning</a>        
    </li>
    <br>
    <li>
         Wikipedia - 
         <a href="https://bit.ly/2kA7wCz">
         ID3 Algorithm</a>        
    </li>
    <br>
    <li>
         Wikipedia - 
         <a href="https://bit.ly/1dm77MT">
         Entropy</a>        
    </li>
    <br>
    <li>
         Wikipedia - 
         <a href="https://bit.ly/2LJbmW3">
         Diversity Index</a>        
    </li>
    <br>
    <li>
         Wikipedia - 
         <a href="https://bit.ly/2J0PoMH">
         Information Gain in Decision Trees</a>        
    </li>
</ul>